## Prorail project: Baseline Model

In [4]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from datetime import timedelta

from sklearn.linear_model import LinearRegression
import scipy.stats as stats
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score 

### Business understanding

##### Achtergrond:

**De doelstelling van het project is om de planners te helpen om de hervatting van het treinverkeer te organiseren. In de praktijk blijkt dat de voorspellingen van de monteurs vaak te pessimistisch zijn. De vraag van ProRail en de planners van de NS is of op basis van de data een betere voorspelling te maken is waardoor de dienstregeling sneller hervat kan worden. De voorspellingen moeten getoond worden in een dashboard waar relevante informatie over de voorspelling, zoals de betrouwbaarheid, getoond wordt.**

Er moet dus gekeken worden naar de tijd voor het oplossen van een probleem. Dit kan gedaan worden door als target het gemiddelde te pakken en daarmee een voorspelling te doen voor aankomende meldingen. We kunnen als vraag hebben:"De reparatietijd is altijd gelijk aan de gemiddelde reparatietijd."

Deze voorspelling moet helpen voor Reizigers, om ze een inschatting te geven hoe lang een probleem gaat duren. Om dat goed te visualiseren wordt eerst de voorspelling gegeven aan een reizigerplanner van de NS/ProRail, want tegenwoordig wordt die voorspelling gedaan met de losse vinger door de aannemers. Deze is meestal erg hoog ingeschat of laag ingeschat, wegens het opmerken/negeren van onverwachtse problemen die kunnen gebeuren. Dat geeft de reiziger veel verwarring en kost veel tijd en geld. Dat moet automatisch en beter kunnen.

##### Doelen:

Om de reparatietijd in te schatten kan je kijken naar de eerder voorkomende meldingen. In dit geval de 0.9 miljoen rijen aan meldingen data. Om daar een beginnetje te maken, gaan we de vraag beantwoorden: "De reparatietijd is altijd gelijk aan de gemiddelde reparatietijd.". Als je de betrouwbaarheid berekend kan je dat gebruiken om accurater de reparatietijd te bepalen.
Toch zal dit op elke plaats en tijd anders zijn dan voorspelt. De plaats kan namelijk lastig bereikbaar zijn of midden in de nacht zijn. Daarnaast werkt de Data ook niet mee. Zo zijn er tijdbepalingen van het bereiken van aannemers, willekeurig ingevoerd, zodat je soms aannemer tijden na de storing tijden krijg waardoor de reparatietijd negatief wordt.

##### Succes ratio:

Dus de programma van eisen in een paar woorden:  
**S**pecifiek: Reparatietijd bepalen voor reizigers  
**M**eetbaar: Gebruik gemaakt van vorige meldingen Data  
**A**cceptabel: Het hebben van een voorspelling geeft de planners voldoening  
**R**ealistisch: Met de juiste Data en ingeschatte waardes is het haalbaar  
**T**ijdsgebonden: Zo snel mogelijk ingeschat worden (rond de 5 min)  

Deze voorspelling moet weergegeven worden in een applicatie (zie Userstory Application)

### Data understanding

Als we de oplostijd willen weten, moeten we beginnen met het hebben van alle tijdsduur van de melding. 

Dus de columns: 
stm_fh_duur (functie duur)
stm_progfh_in_duur (prognose duur)


<!-- We willen de hele process tijd weten totdat het mogelijk is om te rijden vanaf het moment dat het probleem in het systeem wordt vermeld, dus meldtijd is het begin. Het einde moet zijn wanneer de treinen officieel kunnen rijden dus officiele einde storing tijd. 

Dan komen deze columns daaruit:   stm_sap_meld_ddt (meldtijd), stm_sap_storeind_ddt (officiele einde storing tijd), stm_sap_meldtijd (meldtijd), stm_sap_storeindtijd (officiele einde storing tijd)

Deze kolommen hebben dezelfde beschrijving dus de keuze wordt gemaakt bij Data preperation. -->

In [5]:
read_data = pd.read_csv('pro-rails_data.csv')
read_data = read_data.drop_duplicates(subset=['stm_sap_meldtekst_lang', 'stm_sap_mon_meld_ddt'])
oplostijd_data = read_data[['stm_fh_duur', 'stm_progfh_in_duur']]
oplostijd_data.dtypes

d:\users\ruben\appdata\local\programs\python\python37\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (0,11,19,21,24,25,27,32,33,36,37,38,39,40,46,47,48,49,51,53,55,56,60,73,75,78,80,82,83,86,87,91,95,99,102,103,104,107,108,109,110,111,112,113,116,118,119,120,121,122,123,124,129) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


stm_fh_duur           float64
stm_progfh_in_duur     object
dtype: object

Uit de dtypes kan je zien de prognosetijd een object (string waarschijnlijk) is. Wat betekent dat deze kolom niet gebruikt wordt om te rekenen. Dus de overgebleven kolom stm_fh_duur is de beste, want het geeft de duur van de melding (in minuten) weer

In [6]:
oplostijd_data = pd.DataFrame(oplostijd_data['stm_fh_duur'])
oplostijd_data

stm_fh_duur
0               NaN
1               0.0
2              51.0
3              40.0
4               6.0
...             ...
898516       2614.0
898518         25.0
898520         83.0
898522         83.0
898524         19.0

[599184 rows x 1 columns]

In [7]:
gemiddelde_oplostijd = oplostijd_data.mean()
gemiddelde_oplostijd = round(float(str(gemiddelde_oplostijd)[15:25]))
print("Gemiddelde: " + str(gemiddelde_oplostijd) + " minuten")
oplostijd_data['gemiddelde'] = gemiddelde_oplostijd
oplostijd_data

Gemiddelde: 715 minuten


stm_fh_duur  gemiddelde
0               NaN         715
1               0.0         715
2              51.0         715
3              40.0         715
4               6.0         715
...             ...         ...
898516       2614.0         715
898518         25.0         715
898520         83.0         715
898522         83.0         715
898524         19.0         715

[599184 rows x 2 columns]

In [8]:
oplostijd_data = oplostijd_data.dropna()
oplostijd_data = oplostijd_data[oplostijd_data.stm_fh_duur != 0]


In [9]:
oplostijd_data['close_to_mean'] = oplostijd_data['stm_fh_duur'] - gemiddelde_oplostijd
oplostijd_data

stm_fh_duur  gemiddelde  close_to_mean
2              51.0         715         -664.0
3              40.0         715         -675.0
4               6.0         715         -709.0
5               7.0         715         -708.0
6              13.0         715         -702.0
...             ...         ...            ...
898516       2614.0         715         1899.0
898518         25.0         715         -690.0
898520         83.0         715         -632.0
898522         83.0         715         -632.0
898524         19.0         715         -696.0

[482314 rows x 3 columns]

In [10]:
oplostijd_data = oplostijd_data.sort_values(by='close_to_mean', ascending=False)
oplostijd_data

stm_fh_duur  gemiddelde  close_to_mean
732303     527176.0         715       526461.0
624514     525269.0         715       524554.0
669374     524794.0         715       524079.0
694480     205536.0         715       204821.0
879958     181527.0         715       180812.0
...             ...         ...            ...
241013          1.0         715         -714.0
438183          1.0         715         -714.0
738597          1.0         715         -714.0
738599          1.0         715         -714.0
456134          1.0         715         -714.0

[482314 rows x 3 columns]

In [11]:
# source: https://stackoverflow.com/questions/30112202/how-do-i-find-the-closest-values-in-a-pandas-series-to-an-input-number
closest_mean_data = oplostijd_data.iloc[(oplostijd_data['close_to_mean']- 0).abs().argsort()]
closest_mean_data

stm_fh_duur  gemiddelde  close_to_mean
300965        715.0         715            0.0
369153        715.0         715            0.0
584417        715.0         715            0.0
884491        715.0         715            0.0
648724        715.0         715            0.0
...             ...         ...            ...
879958     181527.0         715       180812.0
694480     205536.0         715       204821.0
669374     524794.0         715       524079.0
624514     525269.0         715       524554.0
732303     527176.0         715       526461.0

[482314 rows x 3 columns]

In [13]:
r2_score(closest_mean_data['gemiddelde'], closest_mean_data['stm_fh_duur'])

0.0

In [15]:
mean_squared_error(closest_mean_data['gemiddelde'], closest_mean_data['stm_fh_duur'], squared=False)

4745.11863978905